In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB

In [16]:
data = pd.read_csv('all.csv',header=0,encoding="utf-8")
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人
1,数据分析及应用开发实习生,150-200/天,上海-徐汇,无经验,本科,"['Python', 'excel', 'POWER QUERY', 'POWER AUTO...",上海朵安诺电子商务有限公司,民营,20-99人
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人


In [18]:
# 城市数据处理
data['城市'] = data['地点'].apply(lambda x:str(x).split('-')[0])
# 去除非范围类数据
data = data[data['薪资范围'] != '面议']
data = data[data.薪资范围.str.contains('/天')==False]
data = data[data.薪资范围.str.contains('以下')==False]
# 薪资数据处理
def salary_handle(word):
    num=0
    if word[-1] == '万':
        num = float(word.strip('万')) * 10000
    elif word[-1] == '千':
        num = float(word.strip('千')) * 1000
    return num
data['薪资下限'] = data.薪资范围.apply(lambda x:x.split('-')[0])
data['薪资下限'] = data.薪资下限.apply(lambda x:salary_handle(x))
#表头处理
data.drop_duplicates(inplace=True)
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,10000.0
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7000.0
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6000.0
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,15000.0
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8000.0


In [19]:
#标签设定
data.loc[data['薪资下限']<=10000.0,'label'] = 1
data.loc[data['薪资下限']>10000.0,'label'] = 0
data.head()

,职位名称,薪资范围,地点,工作经验,学历要求,岗位标签,公司名称,公司类型,公司规模,城市,薪资下限,label
0,数据处理工程师,1万-2万,上海-宝山,不限,本科,"['MATLAB', 'Python', '机器学习', '神经网络', 'Shell']",南京思影科技有限公司,民营,20-99人,上海,10000.0,1.0
2,数据分析专员,7千-9千,上海-浦东,3-5年,本科,['销售数据分析'],西安老马识途信息技术有限公司,民营,20-99人,上海,7000.0,1.0
3,财务数据分析,6千-1.2万,上海-宝山,不限,大专,"['Python', '财务数据分析', '运营数据分析']",上海进发建筑工程有限公司,民营,20-99人,上海,6000.0,1.0
4,数据分析师,1.5万-1.8万,上海-浦东,3-5年,本科,"['Python', 'SQL']",武汉景行英才人力资源服务有限公司,民营,100-299人,上海,15000.0,0.0
5,数据分析专员,8千-1.2万,上海-浦东,1-3年,本科,"['SAS', 'SQL']",上汽通用汽车金融,合资,500-999人,上海,8000.0,1.0


In [27]:
data.drop(columns=["职位名称","薪资范围","地点","岗位标签","公司名称"])

,工作经验,学历要求,公司类型,公司规模,城市,薪资下限,label
0,不限,本科,民营,20-99人,上海,10000.0,1.0
2,3-5年,本科,民营,20-99人,上海,7000.0,1.0
3,不限,大专,民营,20-99人,上海,6000.0,1.0
4,3-5年,本科,民营,100-299人,上海,15000.0,0.0
5,1-3年,本科,合资,500-999人,上海,8000.0,1.0
...,...,...,...,...,...,...,...
5137,3-5年,大专,民营,10000人以上,青岛,12000.0,0.0
5138,不限,大专,民营,20-99人,青岛,5000.0,1.0
5139,3-5年,本科,民营,300-499人,青岛,6000.0,1.0
5140,不限,本科,外商独资,500-999人,青岛,15000.0,0.0


In [30]:
data.value_counts('学历要求')

学历要求
本科       3050
大专       1242
硕士        395
学历不限       73
高中         16
博士         11
中专/中技       5
初中及以下       2
中技          1
学历要求        1
dtype: int64

In [31]:
# 学历要求的重新合并，减少类别
data.loc[data['学历要求']=='高中','学历要求'] = '高中及以下'
data.loc[data['学历要求']=='中专/中技','学历要求'] = '高中及以下'
data.loc[data['学历要求']=='初中及以下','学历要求']  = '高中及以下'
data.loc[data['学历要求']=='中技','学历要求']  = '高中及以下'

In [32]:
data.value_counts('公司类型')

公司类型
民营          2316
上市公司         547
国企           512
股份制企业        396
合资           387
外商独资         251
             219
其它           125
事业单位          24
社会团体           7
港澳台公司          5
学校/下级学院        2
律师事务所          2
代表处            1
公司类型           1
医院             1
dtype: int64

In [33]:
data.loc[data['公司类型']=='事业单位','公司类型'] = '其它'
data.loc[data['公司类型']=='社会团体','公司类型'] = '其它'
data.loc[data['公司类型']=='港澳台公司','公司类型']  = '其它'
data.loc[data['公司类型']=='学校/下级学院','公司类型']  = '其它'
data.loc[data['公司类型']=='律师事务所','公司类型']  = '其它'
data.loc[data['公司类型']=='代表处','公司类型'] = '其它'
data.loc[data['公司类型']=='医院','公司类型']  = '其它'
data.loc[data['公司类型']==' ','公司类型']  = '其它'

In [35]:
# 确定编码的字段
one_hot_cols = data.columns.tolist()
# 标签的字段不需要编码
one_hot_cols.remove('label')

data_encode = pd.get_dummies(data, columns=one_hot_cols)

# 连续的特征处理
encoder = LabelEncoder()
# 转换数据类型
data_con = data.astype('str')
# 编码转换
data_encode = data_con.apply(encoder.fit_transform)

print('encode over')
# 数据差分成训练数据，测试数据集。

train = data_encode.loc[0:1000,:]
test = data_encode.loc[1000:, :]

train = train.dropna(axis=0)
test= test.dropna(axis=0)


encode over


In [36]:
# 准备数据集
X_train = train.drop('label',axis=1)
Y_train =train['label'].astype('int64')  # 转换为数值型

X_test = test.drop('label',axis=1)
Y_test =test['label'].astype('int64')  # 转换为数值型

In [37]:
# model
# LogisticRegression
lr_classifier = LogisticRegression(solver='lbfgs',C=1.0,max_iter=10000)
lr_classifier.fit(X_train,Y_train)
train_score = lr_classifier.score(X_train,Y_train)
test_score = lr_classifier.score(X_test,Y_test)
print(F"训练集准确率：{train_score*100:8.2f}% ，测试集准确率：{test_score*100:8.2f}%，逻辑回归")
#逻辑回归
pre_lr = lr_classifier.predict(X_test)
report_lr = classification_report(Y_test,pre_lr)
print(report_lr)
#混淆矩阵
con_lr = confusion_matrix(Y_test,pre_lr)
print(con_lr)

训练集准确率：   89.72% ，测试集准确率：   88.45%，逻辑回归
              precision    recall  f1-score   support

           0       1.00      0.70      0.82      1476
           1       0.84      1.00      0.91      2387

    accuracy                           0.88      3863
   macro avg       0.92      0.85      0.87      3863
weighted avg       0.90      0.88      0.88      3863

[[1031  445]
 [   1 2386]]


In [38]:
#naive bayes
gau_classifier = GaussianNB()
gau_classifier.fit(X_train,Y_train)
train_score = gau_classifier.score(X_train,Y_train)
test_score = gau_classifier.score(X_test,Y_test)
print(F"训练集准确率：{train_score*100:8.2f}% ，测试集准确率：{test_score*100:8.2f}%，朴素贝叶斯分类")
# 朴素贝叶斯分类
pre_gau = gau_classifier.predict(X_test)
report_gau = classification_report(Y_test,pre_gau)
print(report_gau)
#混淆矩阵
con_gau = confusion_matrix(Y_test,pre_gau)
print(con_gau)

训练集准确率：   82.87% ，测试集准确率：   69.07%，朴素贝叶斯分类
              precision    recall  f1-score   support

           0       0.93      0.20      0.34      1476
           1       0.67      0.99      0.80      2387

    accuracy                           0.69      3863
   macro avg       0.80      0.60      0.57      3863
weighted avg       0.77      0.69      0.62      3863

[[ 302 1174]
 [  21 2366]]
